## Spam mail分類を行う

In [40]:
%matplotlib inline
""" spam を分類する　"""
import libs.load_spam_data
import importlib
importlib.reload(libs.load_spam_data)

# データの読み込み
FILE_PATH = './spam_data/spam.csv'
train_tokes, train_labels, test_tokes, test_labels = libs.load_spam_data.load_spam_data(FILE_PATH)
print(f"Train tokens: {len(train_tokes)}")
print(f"Test tokens: {len(test_tokes)}")
print("train #1000:", train_tokes[1000])
print("train label #1000:", train_labels[1000])



[nltk_data] Downloading package punkt to /Users/nisida/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nisida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.0.2
-------------
hum count   =  4825
supam count =  747
-------------
max_token_len =  171
[665, 202, 221, 666, 667, 668, 595, 477, 21, 669, 294, 670, 553]
Train tokens: 4457
Test tokens: 1114
train #1000: [3517, 3518, 647, 2176]
train label #1000: 1


{'name': 1, 'price':2} ->  all_token内の全ての単語にユニークなID（数字)をふる（Pythonの辞書で作成する）. -> コーパスと呼ぶ。

気になるpoint

token1 = [1,3, 6, 7] , token2 = [10, 13, 14, 15 16, 17]


In [ ]:
import nltk 

print(nltk.__version__)